In [11]:
import requests
import json
import numpy as np
import pandas as pd

def yelp_api_request(term, location, query_number):
    """term = 'nightlife' (string)
        location = 'zipcode', 'neighborhood' (string)
        query_number = round of query (1,2,3...) -> determines offset (integer)
    """
    offset = (query_number - 1)*50 + 1 
    api_key='gPWslag3JTblGMOVOLoyWFXn7PwrZ10M5Z4j-Ej54KeW2Svd2vrxdEz_Jpl3ls09mii6uZcl1vD89TwD23DaA3Qj01xQtLsVIzz9HqZge0tjGyUiWvU_lhbb5LzmXXYx'

    url='https://api.yelp.com/v3/businesses/search'
    params={'term': term, 'location': location, 'limit': 50, 'offset': offset}
    headers = {'Authorization': 'Bearer %s' % api_key} 
    req = requests.get(url, params=params, headers=headers)
    json_data = req.json()
    # with open('/Users/cmvandev/Desktop/debug/data.json', 'w', encoding='utf-8') as f:
    #    json.dump(json_data, f, ensure_ascii=False, indent=4) 
    # data = json.load(open('/Users/cmvandev/Desktop/debug/data.json'))
    df = pd.DataFrame(json_data["businesses"])
    business_id = list(df['id'])
    reviews_dataframe = pd.DataFrame()
    list_of_reviews = []
    new = pd.DataFrame(index=business_id, columns = ['rev1', 'rev2', 'rev3'])
    for id_ in business_id:
        list_of_reviews_for_each_business =[]
        url="https://api.yelp.com/v3/businesses/" + id_ + "/reviews"
        req_reviews = requests.get(url, headers=headers)
        #json_data_reviews = req_reviews.json()
        try:
            if req_reviews.json()['reviews'][0]['text']!= None and req_reviews.json()['reviews'][0]['text'] != '' :
                list_of_reviews_for_each_business.append(req_reviews.json()['reviews'][0]['text'])
        except:
            list_of_reviews_for_each_business.append('No Review')

        try:
            if req_reviews.json()['reviews'][1]['text']!= None and req_reviews.json()['reviews'][1]['text'] != '' :
                list_of_reviews_for_each_business.append(req_reviews.json()['reviews'][1]['text'])
        except:
            list_of_reviews_for_each_business.append('No Review')

        try:
            if req_reviews.json()['reviews'][2]['text']!= None and req_reviews.json()['reviews'][2]['text'] != '' :
                list_of_reviews_for_each_business.append(req_reviews.json()['reviews'][2]['text'])
        except:
            list_of_reviews_for_each_business.append('No Review')

        list_of_reviews.append(list_of_reviews_for_each_business)
    #     list_of_reviews_for_each_business.clear()

        new.loc[id_]['rev1'] = list_of_reviews_for_each_business[0]
        new.loc[id_]['rev2'] = list_of_reviews_for_each_business[1]
        new.loc[id_]['rev3'] = list_of_reviews_for_each_business[2]

    # merge business info and reviews dataframe on business ID
    df_merged = df.merge(new, left_on = 'id', right_index = True)

    file_name = '{}_{}_{}_yelp.csv'.format(term, location, query_number)

    #df_merged.to_csv('/Users/cmvandev/Desktop/debug/data_merged.csv')
    df_merged.to_csv('/Users/andrewnorris/Google Drive/capstone_mome/data/'+file_name)

In [32]:
yelp_api_request('nightlife','11221', 5)

In [13]:
df_merged.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price,rev1,rev2,rev3
0,gK-d-vOz9UmXIE_08jEOzg,the-bad-ass-astoria,The Bad Ass,https://s3-media2.fl.yelpcdn.com/bphoto/CwCB39...,False,https://www.yelp.com/biz/the-bad-ass-astoria?a...,4,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",4.0,"{'latitude': 40.77632, 'longitude': -73.91518}",[],"{'address1': '26-17 23rd Ave', 'address2': '',...",+17184408378,(718) 440-8378,786.203723,NaN,This place just opened in Astoria and after tw...,It's fine for what it is. \nThey only serve dr...,Super cute. Friendly staff. Tasty cocktails. D...
1,TI5dWiHdFn55WQDmvffu0Q,the-lingerie-shoppe-aka-astoria-s-secret-queens,The Lingerie Shoppe aka Astoria’s Secret,https://s3-media1.fl.yelpcdn.com/bphoto/QnthqK...,False,https://www.yelp.com/biz/the-lingerie-shoppe-a...,10,"[{'alias': 'speakeasies', 'title': 'Speakeasie...",4.5,"{'latitude': 40.76406, 'longitude': -73.9145}","[pickup, delivery]","{'address1': '28-53 Steinway St', 'address2': ...",,,1798.451666,$$,What an amazing cool alternative to the Astori...,This place is amazing. Come here for a date ni...,What a wonderful experience!! I celebrated my ...
2,iOi6-KC3dyBAE_Bbgokyxw,vibras-restaurant-and-lounge-new-york,Vibras Restaurant & Lounge,https://s3-media3.fl.yelpcdn.com/bphoto/q73cQP...,False,https://www.yelp.com/biz/vibras-restaurant-and...,1,"[{'alias': 'lounges', 'title': 'Lounges'}]",5.0,"{'latitude': 40.76377, 'longitude': -73.87815}",[],"{'address1': '91-01 Astoria Blvd', 'address2':...",+19173964225,(917) 396-4225,2942.732414,NaN,I have known the owners for quite some time at...,No Review,No Review
3,CM8WhuCoXd3j3u4Jjqol-g,savanna-rooftop-long-island-city,Savanna Rooftop,https://s3-media3.fl.yelpcdn.com/bphoto/FgDzcf...,False,https://www.yelp.com/biz/savanna-rooftop-long-...,88,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.0,"{'latitude': 40.75198, 'longitude': -73.94752}",[],"{'address1': '11-01 43rd Ave', 'address2': '',...",+17185689860,(718) 568-9860,4561.894968,$$,This place has GREAT views and fairly priced d...,Went there just in time for the sunset.\nAroun...,My friend and I came on a weekday during happy...
4,O7CnII31QnhLIYoUlOXrZA,la-sala-astoria,La Sala,https://s3-media2.fl.yelpcdn.com/bphoto/DZKzPR...,False,https://www.yelp.com/biz/la-sala-astoria?adjus...,123,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 40.7632006, 'longitude': -73.9286...",[],"{'address1': '3205 Crescent St', 'address2': '...",+17186066121,(718) 606-6121,2564.177719,$$,I been wanting to check this place out for yea...,What a little surprise bar! Very pleasant atmo...,"Absolutely fell in love with this cute, charmi..."


In [150]:
new

,rev1,rev2,rev3
FpgzHQMSNTTjagrKXxOw5A,I'm a believer now. I'm re-reviewing after a f...,I was here on a Friday for dinner. I asked if ...,HEALTH CODE VIOLATION!!!!\nRead !\n\nI am a re...
HSKcCmRFqzqG542Hy_eftw,Moved back into my upper eastside place after ...,The ambience is good. I came here for the lob...,Great family owned Italian gem upper east side...
DEHWHskp2gSAkfdylc4SNA,Favorite place in Harlem! We come for the del...,The food was good. The leg of lamb was flavor...,We had the escargots and seasonal soup (chicke...
OkhbpTKJMb5_osTuynmT9Q,If you live in the neighborhood you know that ...,"The food is really good here, and the drinks f...",Stopped for Sunday lunch after our visit to th...
20LSOx_4K2wghTsUx4gSsg,Came here for my mom's birthday. Took a while ...,I've been here 4 times and all 4 times the foo...,Been to this place a few times already and eve...
XOnspIaW6mYVmrvgqOaYSQ,I have only been here once but I definitely pl...,COVID-19 REVIEW\n\nI'm going to start with som...,I had this spot bookmarked and I was looking f...
M7Bl6U2EjxI-n0ksoYtZ8w,Love the new and improved Vero. I've been to t...,Quick recap: awesome date spot. Good food. Dec...,Maybe this review is a waste of time but it's ...
I0rjVA3Zf2avhX-mNUAKeg,Went for music bingo on Sunday 4-6 and David (...,The turkey club was huge and pretty good. Need...,Note: Wednesdays [formerly] 25-cent wing speci...
mBZUWdasmua9XOsWy8xdWA,A decent place with good brunch options. I ord...,Jacques has upped its game recently. This plac...,I use to live in the UES so as I'm I walking a...
5LqclCP-RTj61-5QNo0jZw,I had a WONDERFUL experience dining here with ...,I've been here multiple times because it's a f...,Don't trust the bar tenders! I've been going ...


In [151]:
len(list_of_reviews)

50

In [37]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,33SyxCFwGOK_ws-05W_-lA,pj-leahys-long-island-city,PJ Leahy's,https://s3-media2.fl.yelpcdn.com/bphoto/X2yX7k...,False,https://www.yelp.com/biz/pj-leahys-long-island...,65,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",3.5,"{'latitude': 40.742645, 'longitude': -73.954452}",[],$$,"{'address1': '5002 Vernon Blvd', 'address2': '...",+17184725131,(718) 472-5131,5711.964603
1,FpgzHQMSNTTjagrKXxOw5A,san-matteo-pizzeria-e-cucina-new-york,San Matteo Pizzeria e Cucina,https://s3-media1.fl.yelpcdn.com/bphoto/SPsfsT...,False,https://www.yelp.com/biz/san-matteo-pizzeria-e...,288,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 40.7746799, 'longitude': -73.95443}","[delivery, pickup]",$$,"{'address1': '1559 2nd Ave', 'address2': '', '...",+12128612434,(212) 861-2434,4058.933767
2,HSKcCmRFqzqG542Hy_eftw,siena-cafe-new-york-3,Siena Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/TsBDNa...,False,https://www.yelp.com/biz/siena-cafe-new-york-3...,191,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 40.78073, 'longitude': -73.953}","[delivery, pickup]",$$,"{'address1': '1580 3rd Ave', 'address2': '', '...",+12124105588,(212) 410-5588,3912.791314
3,DEHWHskp2gSAkfdylc4SNA,yatenga-french-bistro-and-bar-new-york,Yatenga French Bistro & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/kwfY8L...,False,https://www.yelp.com/biz/yatenga-french-bistro...,254,"[{'alias': 'french', 'title': 'French'}, {'ali...",4.0,"{'latitude': 40.8142691169743, 'longitude': -7...","[delivery, pickup]",$$,{'address1': '2269 Adam Clayton Powell Jr Blvd...,+12126900699,(212) 690-0699,5026.567349
4,OkhbpTKJMb5_osTuynmT9Q,lukes-bar-and-grill-new-york-2,Luke's Bar and Grill,https://s3-media1.fl.yelpcdn.com/bphoto/FxymcH...,False,https://www.yelp.com/biz/lukes-bar-and-grill-n...,137,"[{'alias': 'tradamerican', 'title': 'American ...",3.5,"{'latitude': 40.77467, 'longitude': -73.95735}","[delivery, pickup]",$$,"{'address1': '1394 3rd Ave', 'address2': '', '...",+12122497070,(212) 249-7070,4304.602788


In [39]:
type(df['id'])

pandas.core.series.Series

In [40]:
list(df['id'])

['33SyxCFwGOK_ws-05W_-lA',
 'FpgzHQMSNTTjagrKXxOw5A',
 'HSKcCmRFqzqG542Hy_eftw',
 'DEHWHskp2gSAkfdylc4SNA',
 'OkhbpTKJMb5_osTuynmT9Q',
 'iYmTPHpjcb3xtHn8DlkoWQ',
 '20LSOx_4K2wghTsUx4gSsg',
 'XOnspIaW6mYVmrvgqOaYSQ',
 'mBZUWdasmua9XOsWy8xdWA',
 'oRMSGKe78ajjduzHw09Hbw',
 '5LqclCP-RTj61-5QNo0jZw',
 'ldt2JdbJqGrQfB6o8vz18A',
 'kz6WiG4CEiTtnSp1LQ88bw',
 'qD7Ttf8FSL6bplLBwjvXXg',
 'gbtGZtbjaFDgTHs2x6gedQ',
 'penftLPo6kx6qCtbsuyLog',
 'Kyx9QFPJUzlC7dmM8SE1bw',
 'I0rjVA3Zf2avhX-mNUAKeg',
 'vJXswY5u9rcMRrYZukpMbQ',
 'RkcQl4uEJWblWf4xX8ssqQ',
 'P_WcaYX89ZmIE4nNVTqS9w',
 'FLRx5tOBMMDJHEUYAAfa8A',
 'l_hBcWIFx_ausYBxLkSglQ',
 'kZysScUNGTaKRjOeMRlxag',
 'j6fSechU9HxzbnK-kjQx1Q',
 'OAqsZMbjv66nOMqlimnYuQ',
 'V7_IQ1ab69JilAlgrSnMXg',
 'jhFqstTMRBAdBHgoQ1nSTA',
 'EgxnURDBkeKXV9rKzHrJLA',
 'Vm6_cXz8hbvZhot0m1aaRg',
 'Trw7ZZlMjlSLqa0owjhxhA',
 'iV5yy8EbjsPoYWtjRhB61Q',
 'AevRIon-_YHQ2FCO-L120w',
 'wpye6v-2fIx7Z2Sz12XubA',
 '8p1HTzXEcC3TCbLz59xMng',
 '4FtqGBB96denVJu8KnuGKg',
 'dazRB9xf_v7xLD2UzCoLrw',
 

In [5]:
print(data_reviews.keys())

dict_keys(['reviews', 'total', 'possible_languages'])


In [17]:
test = pd.DataFrame()

In [18]:
type(test)

pandas.core.frame.DataFrame

In [1]:
 new = pd.DataFrame(index=business_id, columns = ['rev1', 'rev2', 'rev3'])

NameError: name 'pd' is not defined

In [56]:
new

,rev1,rev2,rev3
FpgzHQMSNTTjagrKXxOw5A,NaN,NaN,NaN
HSKcCmRFqzqG542Hy_eftw,NaN,NaN,NaN
DEHWHskp2gSAkfdylc4SNA,NaN,NaN,NaN
OkhbpTKJMb5_osTuynmT9Q,NaN,NaN,NaN
20LSOx_4K2wghTsUx4gSsg,NaN,NaN,NaN
XOnspIaW6mYVmrvgqOaYSQ,NaN,NaN,NaN
M7Bl6U2EjxI-n0ksoYtZ8w,NaN,NaN,NaN
I0rjVA3Zf2avhX-mNUAKeg,NaN,NaN,NaN
mBZUWdasmua9XOsWy8xdWA,NaN,NaN,NaN
oRMSGKe78ajjduzHw09Hbw,NaN,NaN,NaN


In [69]:
new.head()

,rev1,rev2,rev3
HSKcCmRFqzqG542Hy_eftw,NaN,NaN,NaN
DEHWHskp2gSAkfdylc4SNA,NaN,NaN,NaN
OkhbpTKJMb5_osTuynmT9Q,NaN,NaN,NaN
20LSOx_4K2wghTsUx4gSsg,NaN,NaN,NaN
XOnspIaW6mYVmrvgqOaYSQ,NaN,NaN,NaN


In [73]:
new.iloc[0][0] = 'text'

In [74]:
new.head()

,rev1,rev2,rev3
HSKcCmRFqzqG542Hy_eftw,text,NaN,NaN
DEHWHskp2gSAkfdylc4SNA,NaN,text,NaN
OkhbpTKJMb5_osTuynmT9Q,NaN,NaN,NaN
20LSOx_4K2wghTsUx4gSsg,NaN,NaN,NaN
XOnspIaW6mYVmrvgqOaYSQ,NaN,NaN,NaN


In [76]:
new.loc['DEHWHskp2gSAkfdylc4SNA']['rev1'] = 'text'
new.head()

,rev1,rev2,rev3
HSKcCmRFqzqG542Hy_eftw,text,NaN,NaN
DEHWHskp2gSAkfdylc4SNA,text,text,NaN
OkhbpTKJMb5_osTuynmT9Q,NaN,NaN,NaN
20LSOx_4K2wghTsUx4gSsg,NaN,NaN,NaN
XOnspIaW6mYVmrvgqOaYSQ,NaN,NaN,NaN
